In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import os
from dotenv import load_dotenv
import logging

logging.basicConfig(filename='logfile.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def log_message(message):
    logging.info(message)

# Função para ajustar o valor
def ajustar_valor(valor_str):
    try:
        valor_str = str(valor_str).strip()
        valor_str = valor_str.replace(',', '.')
        valor_float = float(valor_str)
        if '.' in valor_str:
            decimal_part = valor_str.split('.')[-1]
            if len(decimal_part) == 1 or (len(decimal_part) > 1 and decimal_part[1] == '0'):
                valor_float *= 10
        return valor_float
    except ValueError:
        return float('nan')

caixa_mapping = {
    'VL_UPANEMA E MOSSORÓ_ITAÚ [CONTA CORRENTE]': '3490',
    'SECRETARIA DE MISSÕES_ESPÉCIE': '3496',
    #COLOQUE OS CAIXAS AQUI
}


load_dotenv()



tabela = pd.read_excel("saida_lancamento.xlsx")
tabela['Data'] = pd.to_datetime(tabela['Data'], dayfirst=True, errors='coerce').dt.strftime('%d/%m/%Y')
tabela['Valor'] = tabela['Valor'].apply(ajustar_valor)
tabela['Data de Pagamento'] = pd.to_datetime(tabela['Data de Pagamento'], dayfirst=True, errors='coerce').dt.strftime('%d/%m/%Y')

navegador = webdriver.Chrome()
wait = WebDriverWait(navegador, 20)

navegador.get('https://www.verbodavida.info/apps/index.php')

usuario_input = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login"]/div[1]/div/input')))
usuario_input.send_keys("")
#COLOQUE EMAIL
senha_input = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login"]/div[2]/div/input')))
senha_input.send_keys("")
#COLOQUE SENHA
login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@name='submit' and text()='Entrar']")))
login_button.click()
time.sleep(4)

soma_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'SOMA')]")))
soma_button.click()
time.sleep(5)

navegador.get('https://www.verbodavida.info/IVV/?mod=ivv&exec=entradas_saidas_dados')
wait.until(EC.url_to_be('https://www.verbodavida.info/IVV/?mod=ivv&exec=entradas_saidas_dados'))
time.sleep(1)

lançamentos_concluídos = False

def processar_linha(index, linha):
    global lançamentos_concluídos
    try:
        plano_de_contas = linha['Plano de Contas']
        centro_custo = linha['Centro de Custo']
        nota_fiscal = linha.get('Nota Fiscal', '')
        descricao = linha['Descrição']
        data_vencimento = linha['Data']
        valor = linha['Valor']
        data_pagamento = linha['Data de Pagamento']
        forma_pagamento = linha['Forma de Pagamento']
        numero_documento = str(linha['Documento'])
        caixa = linha['Caixa']

        log_message(f"Selecionando plano de contas: {plano_de_contas}")
        
        select_element = WebDriverWait(navegador, 10).until(
        EC.visibility_of_element_located((By.ID, 'plano')))
        select = Select(select_element)
        select.select_by_visible_text(plano_de_contas)
        
        log_message(f"Plano de contas '{plano_de_contas}' selecionado com sucesso.")

    
        time.sleep(1)

        log_message(f"Selecionando centro de custo: {centro_custo}")
        select_custo_element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="exampleStandardForm"]/div[7]/div/select')))
        select_custo = Select(select_custo_element)
        select_custo.select_by_visible_text(centro_custo)
        log_message(f"Centro de custo '{centro_custo}' selecionado com sucesso.")
        time.sleep(1)

        if pd.notna(nota_fiscal):
            try:
                log_message(f"Preenchendo nota fiscal: {nota_fiscal}")
                nota_fiscal_input = wait.until(EC.element_to_be_clickable((By.NAME, 'nf')))
                nota_fiscal_input.send_keys(str(nota_fiscal))
                log_message("Nota fiscal preenchida com sucesso.")
            except Exception as e:
                log_message(f"Erro ao preencher a nota fiscal: {e}")
                tabela.loc[index, 'Erro'] = f"Erro ao preencher a nota fiscal: {e}"
                return False
        time.sleep(1)

        log_message(f"Preenchendo descrição: {descricao}")
        descricao_input = wait.until(EC.element_to_be_clickable((By.NAME, 'descricao')))
        descricao_input.send_keys(descricao)
        log_message("Descrição preenchida com sucesso.")
        time.sleep(2)

        log_message(f"Preenchendo data de vencimento: {data_vencimento}")
        data_vencimento_input = wait.until(EC.element_to_be_clickable((By.NAME, 'data_vencimento')))
        data_vencimento_input.send_keys(data_vencimento)
        log_message("Data de vencimento preenchida com sucesso.")
        time.sleep(2)

        log_message(f"Preenchendo valor: {valor}")
        valor_input = wait.until(EC.element_to_be_clickable((By.NAME, 'valor')))
        valor_input.send_keys(valor*-1)
        log_message("Valor preenchido com sucesso.")
        time.sleep(2)

        try:
            button_enviar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_salvar"]')))
            button_enviar.click()
            log_message("Botão salvar clicado com sucesso.")
        except Exception as e:
            log_message(f"Erro ao clicar no botão salvar: {e}")
            tabela.loc[index, 'Erro'] = f"Erro ao clicar no botão salvar: {e}"
            return False
        time.sleep(2)

        try:
            button_sim = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@id="button-0" and text()="Sim"]')))
            button_sim.click()
            log_message("Botão 'Sim' clicado com sucesso.")
        except Exception as e:
            log_message(f"Erro ao clicar no botão 'Sim': {e}")
            tabela.loc[index, 'Erro'] = f"Erro ao clicar no botão 'Sim': {e}"
            return False
        time.sleep(2)

        try:
            button_inserir_pagamento = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/div/div/form/div[29]/div/div[2]/a')))
            button_inserir_pagamento.click()
            log_message("Botão 'Inserir Pagamento' clicado com sucesso.")
        except Exception as e:
            log_message(f"Erro ao clicar no botão 'Inserir Pagamento': {e}")
            tabela.loc[index, 'Erro'] = f"Erro ao clicar no botão 'Inserir Pagamento': {e}"
            return False
        time.sleep(1)

        log_message(f"Preenchendo data de pagamento: {data_pagamento}")
        data_pagamento_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[5]/div/div/form/div[2]/div/div/div[1]/div[1]/div/input')))
        navegador.execute_script("arguments[0].value = '';", data_pagamento_input)  # Limpa o campo antes de inserir
        data_pagamento_input.send_keys(data_pagamento)
        log_message("Data de pagamento preenchida com sucesso.")
        time.sleep(1)

        
        log_message(f"Selecionando forma de pagamento: {forma_pagamento}")
        forma_pagamento_select = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[5]/div/div/form/div[2]/div/div/div[2]/div[1]/div/select')))
        Select(forma_pagamento_select).select_by_visible_text(forma_pagamento)
        log_message(f"Forma de pagamento '{forma_pagamento}' selecionada com sucesso.")
        time.sleep(1)

        if forma_pagamento != "DINHEIRO":
            log_message(f"Preenchendo número do documento: {numero_documento}")
            numero_documento_input = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[5]/div/div/form/div[2]/div/div/div[2]/div[3]/div/input')))
            numero_documento_input.send_keys(numero_documento)
            log_message("Número do documento preenchido com sucesso.")
            time.sleep(1)
        else:
             pass


        log_message(f"Selecionando caixa: {caixa}")
        log_message(f"Caixa '{caixa}' selecionado com sucesso.")

        select_caixa = WebDriverWait(navegador, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="form_pagamentos"]/div[2]/div/div/div[4]/div[2]/div/select')))
        select_3 = Select(select_caixa)

        if caixa in caixa_mapping:
            select_3.select_by_value(caixa_mapping[caixa])
        else:
            print(f"Valor de caixa '{caixa}' não encontrado no mapeamento.")
        time.sleep(1)



        try:
            button_salvar_pagamento = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[5]/div/div/form/div[3]/button')))
            button_salvar_pagamento.click()
            log_message("Botão salvar pagamento clicado com sucesso.")
        except Exception as e:
            log_message(f"Erro ao clicar no botão salvar pagamento: {e}")
            tabela.loc[index, 'Erro'] = f"Erro ao clicar no botão salvar pagamento: {e}"
            return False
        time.sleep(1)

        try:
            button_okay = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/button[1]')))
            button_okay.click()
            log_message("Botão OKAY clicado com sucesso.")
        except Exception as e:
            log_message(f"Erro ao clicar no botão OKAY: {e}")
            tabela.loc[index, 'Erro'] = f"Erro ao clicar no botão OKAY: {e}"
            return False
        time.sleep(1)

        id_lancamento = navegador.current_url.split("ID=")[-1]
        tabela.loc[index, 'Id do lancamento'] = id_lancamento

        lançamentos_concluídos = True
        return True

    except Exception as e:
        log_message(f"Erro ao processar a linha {index}: {e}")
        tabela.loc[index, 'Erro'] = f"Erro ao processar a linha {index}: {e}"
        return False

for index, linha in tabela.iterrows():
    sucesso = processar_linha(index, linha)
    if not sucesso:
        log_message("Processo interrompido devido ao erro.")
        break
    navegador.get('https://www.verbodavida.info/IVV/?mod=ivv&exec=entradas_saidas_dados')
    wait.until(EC.url_to_be('https://www.verbodavida.info/IVV/?mod=ivv&exec=entradas_saidas_dados'))
    time.sleep(1)

if lançamentos_concluídos:
    nome_arquivo = f"saida_lancamento_atualizado_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
    tabela.to_excel(nome_arquivo, index=False)
    log_message(f"Planilha salva como {nome_arquivo}.")
else:
    log_message("Nenhum lançamento foi concluído com sucesso. Planilha não foi salva.")
